In [61]:
import duckdb

conn = duckdb.connect("data/data.db")
conn.sql("select count(*) from reid_cluster where not is_bad_frame")


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        11598 │
└──────────────┘

In [36]:
import numpy as np

query = """
with data as (
    select
        d.frame_id,
        d.id,
        d.osnet_embedding,
        f.n_objects_detected,
        f.is_irregular,
        d.conf
    from detection d
    left join frame f on d.frame_id=f.id
    where f.video_id = '5735c376-9c25-4620-8b3f-2ec4e5cc9f55'
)

select
    *
from data
"""

df = conn.sql(query).df()

df['conf_under_p5'] = df['conf'] < np.quantile(df['conf'].to_numpy(), 0.05)
df = df[df.apply(lambda row: not (row['conf_under_p5'] or row['is_irregular']), axis=1)]

df.head()

,frame_id,id,osnet_embedding,n_objects_detected,is_irregular,conf,conf_under_p5
0,182b6232-2853-4c82-975f-8d92545a9d13,4d1565d1-e4df-4062-80e6-439e6060cbc7,"[0.0, 0.1531789, 0.0, 0.0, 0.006920417, 0.0123...",1,False,0.819785,False
1,8b222e63-e5d0-4a6c-be6f-c47f0c810820,7e3016de-5a4a-4527-a516-539f778c99dd,"[0.0058682356, 0.15271908, 0.0, 0.0, 0.0, 0.01...",1,False,0.826776,False
2,16fcfdd9-6d1e-4cd9-b193-64758202855e,8966d0de-1273-4080-afbf-18c7f9123a81,"[0.021453246, 0.16050723, 0.0, 0.0, 0.00687443...",1,False,0.835311,False
3,b17bd0ad-304e-4076-8b0b-dea05d3ce819,b64d1ae5-0c96-413b-b70b-406880592270,"[0.019822454, 0.15694407, 0.0, 0.0, 0.01747229...",1,False,0.830362,False
4,f747106c-4d22-43e0-bccb-69384499fbee,47ae2586-6949-4c00-8e59-1367f6403dd7,"[0.017267754, 0.13731705, 0.0, 0.0, 0.01530893...",1,False,0.831142,False


In [41]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

pca_50 = PCA(n_components=50).fit_transform(df['osnet_embedding'].to_list())
df['cluster_id_raw'] = KMeans(n_clusters=2).fit_predict(pca_50)
len(df)

/Users/besaleli/reid_mvp/.venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:335: RuntimeWarning: divide by zero encountered in matmul
  Q, _ = normalizer(A @ Q)
/Users/besaleli/reid_mvp/.venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:335: RuntimeWarning: overflow encountered in matmul
  Q, _ = normalizer(A @ Q)
/Users/besaleli/reid_mvp/.venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:335: RuntimeWarning: invalid value encountered in matmul
  Q, _ = normalizer(A @ Q)
/Users/besaleli/reid_mvp/.venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:336: RuntimeWarning: divide by zero encountered in matmul
  Q, _ = normalizer(A.T @ Q)
/Users/besaleli/reid_mvp/.venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:336: RuntimeWarning: overflow encountered in matmul
  Q, _ = normalizer(A.T @ Q)
/Users/besaleli/reid_mvp/.venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:336: RuntimeWarning: invalid value encountered in matmul
  Q, _ = normaliz

4819

In [52]:
conn.register("clusters", df)
query = """
with cpf as (
    select
        frame_id,
        count(distinct cluster_id_raw) as n_distinct_clusters_per_frame
    from clusters
    group by 1
    order by 1
),

bad_frames as (
    select
        clusters.frame_id
    from clusters
    left join cpf on clusters.frame_id = cpf.frame_id
    where
        n_objects_detected > 1
        and n_objects_detected > n_distinct_clusters_per_frame
    qualify count(*) over (partition by clusters.frame_id) > 1
),

cleaned as (
    select
        *,
        frame_id in (select * from bad_frames) as is_bad_frame,
        case
            when frame_id in (select * from bad_frames) then -1
            else cluster_id_raw
        end as cluster_id
    from clusters
)

select
    uuidv4() as id,
    cleaned.id as person_detection_id,
    cleaned.cluster_id,
    cleaned.is_bad_frame
from cleaned
left join detection on cleaned.id=detection.id
"""

clusters = conn.sql(query).df()
conn.close()